In [1]:
pwd

'/home/jupyter/document_ai_production'

In [2]:
!ls -sh

total 44K
8.0K DOCUMENT_AI_WORKING_JULY_2022.ipynb
8.0K document_ai_Working_GAMMA_process_json_documentai_response.ipynb
 12K log_documentai--08--53--07--4_7_2022.log
4.0K log_documentai--09--08--39--4_7_2022.log
4.0K log_documentai--09--10--24--4_7_2022.log
4.0K log_documentai--10--12--06--4_7_2022.log
4.0K log_documentai--10--12--50--4_7_2022.log


In [13]:
import pandas as pd

In [11]:
import json

In [12]:
import regex as re

In [3]:
from datetime import datetime
def time_now():
    '''Get Current Time'''
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S.%f")
    print("Current Time =", current_time)
    return now


############
print('Starting...')
start = time_now()
############


Starting...
Current Time = 10:13:22.169048


In [4]:
def create_file_name_time(file_prefix = 'NLP_RESULTS', file_time='datetime object'):
    file_name = f'{str(file_time.hour).zfill(2)}--{str(file_time.minute).zfill(2)}--{str(file_time.second).zfill(2)}--{file_time.day}_{file_time.month}_{file_time.year}'
    file_name = f'{file_prefix}--{file_name}'
    return file_name

In [5]:

project_id = 'lexical-period-346812'.strip()
location = 'us'
processor_id = 'a23e259771b350c3'.strip()
bucket_name = 'sheikh-fitzgerald'.strip()
input_prefix = 'document_ai_input/'.strip()
output_prefix = 'document_ai_just_made_it_now/'.strip()

from google.cloud import documentai_v1 as documentai
from google.cloud.documentai_v1.types import document


In [6]:
log_file_name =  create_file_name_time('log_documentai', start)

In [7]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format='{asctime} {levelname:<8} {message}',
    style='{',
    filename=f'{log_file_name}.log',
    filemode='w')

In [8]:
logging.debug(f'{input_prefix}')
logging.debug(f'{output_prefix}')

In [9]:
def setup_and_scan():
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}
    docai = documentai.DocumentProcessorServiceClient(client_options=opts)

    # define the documents to scan in
    # input_files = documentai.GcsPrefix(gcs_uri_prefix=input_prefix)
    input_files = documentai.GcsPrefix(gcs_uri_prefix=f"gs://{bucket_name}/{input_prefix}")
    input_config = documentai.BatchDocumentsInputConfig(gcs_prefix=input_files)
    
    logging.debug(f'INPUT_FILES: {input_files}')
    logging.debug(f'INPUT_CONFIG: {input_config}')

    # define the output location
    destination_uri = f"gs://{bucket_name}/{output_prefix}"
    output_config = documentai.DocumentOutputConfig(gcs_output_config={"gcs_uri": destination_uri})
    
    logging.debug(f'{"destination_uri".upper()}: {destination_uri}')
    logging.debug(f'{"output_config".upper()}: {output_config}')

    # define and submit the request
    request = documentai.BatchProcessRequest(
        name=f"projects/{project_id}/locations/{location}/processors/{processor_id}",
        input_documents=input_config,
        document_output_config=output_config
    )
    result = docai.batch_process_documents(request)
    
    print(f"RESULT IN BUCKET: {destination_uri}")
    return destination_uri


In [10]:
ocr_folder = setup_and_scan()

RESULT IN BUCKET: gs://sheikh-fitzgerald/document_ai_just_made_it_now/


In [15]:
!gsutil cp -r gs://sheikh-fitzgerald/document_ai_just_made_it_now/ . 

Copying gs://sheikh-fitzgerald/document_ai_just_made_it_now/9128179475080842369/0/document_ai_sample-0.json...
/ [1 files][  6.1 MiB/  6.1 MiB]                                                
Operation completed over 1 objects/6.1 MiB.                                      


In [18]:
pwd()

'/home/jupyter/document_ai_production'

In [19]:
with open('./document_ai_just_made_it_now/9128179475080842369/0/document_ai_sample-0.json' , encoding='utf-8') as jf:
    data = json.load(jf)

In [23]:
def blocks_text(data_):
    text_ = data_['text']
    container = []
    for page in data_['pages']:
        for k,v in page.items(): # dict
            for layout in page['blocks']:
                container.append(
                layout_to_text(layout, text_)
                )
    return container


def layout_to_text(layout, text_):
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirity of the document's text. This function converts
    offsets to a string.
    """
    response = ""
    for segment in layout['layout']['textAnchor']['textSegments']:
        start_index = int(segment.get('startIndex', "0"))
        end_index = int(segment['endIndex'])
        response += text_[start_index:end_index]

    return response

In [24]:
import os

In [25]:
os.getcwd()

'/home/jupyter/document_ai_production'

In [28]:
from google.cloud import storage

bucket_name = bucket_name
prefix = output_prefix
dl_dir = os.getcwd()

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs:
    filename = blob.name.replace('/', '_') 
    blob.download_to_filename(dl_dir + filename)  # Download